# Show Summary Statistics

In this notebook, we will demonstrate how to process data that we saved in CSV format in the previous notebook. You will learn how to calculate the frequency of individual columns in a CSV table and determine which author appears most frequently in the database for a given year.

This notebook is suitable for both beginners and those who want to familiarize themselves with data processing in Python.

## Requirements

To work with this notebook, you need to have a CSV file created from the "Extract Data" notebook.

## Prerequisites

This notebook does not require deep knowledge of Python, but a basic understanding of programming will be helpful.

## Notebook Structure

The notebook is divided into several sections:

0. **Preparation**: We will import the necessary libraries for processing the CSV file.

1. **Loading from CSV**: We will show you how to load our data stored in CSV format.

2. **Summary Statistics**: You will learn how to extract information from the data, such as the most frequent authors.

## Additional Resources

- [LearnPython.org](https://www.learnpython.org/): This online course offers Python tutorials for both beginners and advanced learners. It can be a useful resource for those looking to expand their Python knowledge.

- [W3Schools.com/Python](https://www.w3schools.com/python/): An extensive tutorial that covers Python along with some popular Python libraries.


### 0. Preparation
First, we need to install the libraries we will be working with. Libraries are packages of functions that are not part of the Python language's core. <br>
To install libraries, use the command `%pip install <library_name>`. Then, we add them to our notebook using the command `import <library_name> (as alias)`. To access functions from the library, use `library_name.function_name` <br>
If we only want to use a single function from a library, we add it using `from <library_name> import <function_name>`.

In [ ]:
# Install libraries
%pip install pandas 
%pip install numpy 
%pip install matplotlib

# Add libraries 
from collections import Counter
import pandas as pd
import numpy as np

### 1. Loading from CSV

First, we use the `pandas` library to load our saved CSV data into a DataFrame data structure (similar to an Excel table). Rows in the DataFrame represent individual records, columns represent different types of data (e.g., author names).
Some fields and subfields may repeat. In the CSV they are concatenated with semicolons.

In [ ]:
# Select base
base = 'cle'

# Path to MARC file
csv_data = 'data/csv/out_{base}.csv'.format(base = base)

# Load CSV file
df = pd.read_csv(csv_data, delimiter=',')

print("Data loaded to DataFrame df.")

In [ ]:
# Print the last 5 records in the DataFrame 'df'
df.tail()

Some values are joined by semicolons, for example, 'figures,' 'description,' and 'genre' in row 9611. Using the `split()` function, we split these values and convert them into a list. If there's no value (i.e., it's NaN), we add an empty list.

In [ ]:
# Iterate through column in the DataFrame
for column in df.columns:
    # There is only one publication date, so we don't need to split this column
    if column != 'year': 

        # Split joined values into a list 
        df[column] = df[column].apply(lambda x: x.split(';') if isinstance(x, str)  else [])

# Print the last 5 records in the DataFrame 'df'
df.tail() 

### 2. Summary Statistics

Let's explore how to calculate simple statistics from the data.

#### 2.1 Counting Occurrences

If we want to know how many times a particular value appears in the database, we can use the `value_counts()` function. In the following example, we count how many records there are from each year in the database.<br>
By using the `records_to_print` parameter, we limit the number of displayed records to the top x values. The values are shown in descending order.


In [ ]:
records_to_print = 20

# Get the value counts of the 'year' column and print the first x records
df['year'].value_counts()[:records_to_print] 

We can see that the majority of records are from the 1980s, followed by the 1970s.

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Instead of the 'year' column, we can display, for example, the 'author' column to find out how many times each author appears in the database.<br>
</div>


#### 2.2 Histogram

We can also visualize the frequency using a histogram. To do this, we use the `plot()` function to plot the values.

First, we need to check if all the values in the 'year' column are numbers. Some databases (like samizdat) contain records where the exact publication year is unknown, and missing digits are replaced with the letter 'u'. We cannot plot these values, so we need to filter them out.<br>
If the 'year' column contains non-numeric values, we will filter them out. We do this using the `isnumeric()` function, which tells us whether a value is a number. Then, we convert the entire column to the integer data type.<br>
If all the values are already numeric, we simply plot them using the `plot()` function. 

We can specify the type of display (histogram) with the `kind` parameter. Additionally, we choose the data to be displayed with the `column` parameter. We can also set the number of bins with the `bins` parameter.

The `plot()` function has other parameters as well, which you can find listed on [this page](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.plot.html).


In [ ]:
# If publication date column is not numeric
if df['year'].dtype != 'int64':

    # Filter non numeric values
    df_years = df[df['year'].str.isnumeric()]

    # Convert string to integer
    df_years['year'] = df_years['year'].astype(int)

else:
    
    df_years = df

# Call plot function and add parameters
df_years.plot(kind = 'hist',column='year',  bins=len(df_years['year'].unique()), grid=False, figsize=(12,8), color='#86bf91', rwidth=0.9, title = "Number of records per year")

#### 2.3 Most Common Author in a Given Year

Now, let's find out which author appeared most frequently as the record's author in a specific year. There are several approaches to this. We will explore three of them.

##### 2.2.1 First Option - value_counts

In our first option, we use the `value_counts()` function that we have already learned. <br>
We use string formatting for nicer output. We specify the format in which we want to display the values and the spacing between them in advance.

For some records, the most common value is an empty list (meaning there is no author in the records). Therefore, for this example, we filter out empty rows.

Then, we iterate through the years of publication in the records. We select rows with a single publication year from our table. We call the `value_counts()` function on them. This function returns frequency counts in descending order, so we only need to take the first value.

<b>The following code only works for columns where values do not repeat within a single row. These columns represent MARC fields that do not repeat.</b>


In [ ]:
# Select column we want to print
picked_column = 'author'

# Format our string
string = "{year: >25} {name: >25} {records: >25}"

# Print the most common values
print(string.format(year = "Year", name = "Most common author", records = "Number of records"))
print("----------------------------------------------------------------------------------------------------")

# Filter out empty values
data_filtered = df[df[picked_column].apply(lambda x: len(x) > 0)]

# Iterate unique years
for year in  np.unique(df['year']):

    # Select only one year 'year'
    if any(data_filtered[data_filtered['year'] == year]['author']):

        # Call function value_counts() and find out the most common author
        value_counts = data_filtered[data_filtered['year'] == year]['author'].value_counts() 

        # Print only first value - the most common author
        s = string.format(year = year, name = ' '.join(value_counts.index[0]), records = str(value_counts.values[0])) 
        print(s)

We can observe the significant variation in the number of articles written for each year in the table.

#### 2.2.2 Second Option - Groupby and Counter

In the second option, we show how to manipulate data using the `groupby()` and `Counter()` functions.

Having our data stored in a DataFrame allows us to use the pandas library functions. In our case, we use the `groupby()` function, which groups data by a selected column.
We also use the `Counter()` function to count the occurrences of each element. Lastly, we use a `lambda` function.


In [ ]:
# Select column 
picked_column = 'author'

# Filter empty values
data_filtered = df[df[picked_column].apply(lambda x: len(x) > 0)]

# Group data by year, save only the most common element in that year 
most_common = data_filtered.groupby('year')[picked_column].apply(lambda x: Counter(element for elements in x for element in elements).most_common()[0] if len(x) > 0 else None) 

# Format our string
string = "{year: >25} {name: >25} {records: >25}"
# Print the most common authors
print(string.format(year = "Year", name = "Most common author", records = "Number of records"))
print("----------------------------------------------------------------------------------------------------")

for key, value in most_common.items():
    print(string.format(year = key, name = value[0], records = value[1]))        

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Try changing the 'picked_column' variable to 'figures'. It finds out what was written about the most in each year.
</div>

##### 2.2.1 Third Option - Custom Functions

As the final option, we write the algorithms ourselves. We implement two functions that we then call. 

The first function, `count_years(df, column)`, calculates the number of articles for each author in each year. It takes a DataFrame `df` and the selected column `column` as inputs. It returns the results as a dictionary where the key is a tuple (year, author's name) and the value is the number of articles.

The second function, `find_most_common(count)`, then determines, for each year, which author wrote the most articles. The input is the dictionary `count` returned by the `count_years(df, column)` function. The function iterates through all the keys in the `count` dictionary and for each year, it keeps track of the author with the highest number of records. Finally, the function returns a sorted dictionary where the key is the year and the value is a tuple (author's name, number of records).


In [ ]:
# Function that counts the number of records in each year for each author
def count_years(df, column):
    
    # Dictionary where keys are tuples (year, author)
    # and values are the number of records for the author in that year
    count = {}
    
    # Iterate rows in the DataFrame
    for _,row in df.iterrows():
        year = row['year']
        for element in row[column]:
            key = (year, element) 
            # If the key exists, add one to the count
            if key in count.keys():
                count[key] += 1
            else:# Key does not exist yet for this year
                # Create the key and set the count to one
                count[key] = 1  
    return count              

# Function that finds the most common author in each year
def find_most_common(count):
       
    # Dictionary, where keys are years
    # and values are (author, number of records) 
    most_common = {}

    # Iterate through keys and values in the dictionary
    for key,value in count.items():
        year = key[0]

        # If the key (year) exists, 
        # check if the value (number of records) is higher than the saved one   
        if year in most_common:
            if most_common[year][1] < value:
                # Remember only the highest value
                most_common[year] = (key[1], value)    
        else:
            # Remember the first value
            most_common[year] = (key[1], value) 
    
    # Sort keys in ascending order         
    years = list(most_common.keys())
    years.sort()
    sorted_most_common = {i: most_common[i] for i in years}        
    return sorted_most_common            

print("Functions saved.")
  

Now we just need to call the functions and print the results. <br>
We use the same formatting that we have already used.

In [ ]:
# Select column
picked_column = 'author'

# Call our functions
count = count_years(df, picked_column)

most_common = find_most_common(count)

# Format string
string = "{year: >25} {name: >25} {records: >25}"

# Print the most common authors
print(string.format(year = "Year", name = "Most common author", records = "Number of records"))
print("----------------------------------------------------------------------------------------------------")
for key, value in most_common.items():
    print(string.format(year = key, name = value[0], records = value[1]))        


It's possible that the tables may slightly differ from each other in cases where the database for a given year contains multiple authors with the same number of records.

#### 2.3 Most Common Author in a Given Year - Repeated Values

As we've seen, it's possible that the frequency of some authors for a given year is the same - meaning that the year contains multiple authors with the same number of records. We write code that handles this situation, and in case of repetition displays all authors. <br>
We demonstrate two options.

#####  2.3.1 First Option - Counter
We create a function `max_counter(lst)` that takes a list `lst` as input. First, we use `Counter` to count the occurrences of individual elements in the list. Then, we find the maximum occurrence count and create a list of elements that have this maximum count. The function returns this list of elements along with their occurrence count.

In [ ]:
def max_counter(lst):

    # Count occurrences of elements in all nested lists 
    counted_elements = Counter(element for elements in lst for element in elements)

    # Find the max value 
    max_val = max(counted_elements.values())

    # Save list of tuples where key is an author and value is number of records
    # Save only the most occurring authors
    ret = [(key, value) for key, value in counted_elements.items() if value == max_val]
    return ret

We iterate through unique years in our data and select the rows where the year matches. Then, we call our prepared function on the selected rows.

In [ ]:

# Select column
picked_column = 'author'

# Filter empty values
data_filtered = df[df[picked_column].apply(lambda x: len(x) > 0)]

# Format string
string = "{year: >25} {name: >25} {records: >25}"

# Print the most common authors
print(string.format(year = "Year", name = "Most common authors", records = "Number of records"))
print("----------------------------------------------------------------------------------------------------")

# Iterate through unique years
for year in np.unique(data_filtered['year']):

    # Call function 'max_counter' on data from one year
    ret = max_counter(data_filtered[data_filtered['year'] == year][picked_column])

    # Iterate through list of most common authors  
    for name, number_of_records in ret:    
        print(string.format(year = year, name = name, records = number_of_records))

##### 2.3.1 Second Option - Adjusting Our Functions

We already have the functions `count_years(df, column)` and `find_most_common(count)` written. In this example, we slightly modify the `find_most_common(count)` function to remember both author names in case we encounter authors with the same number of articles as those already saved.

In [ ]:
# Function that finds the most common author in each year
def count_years(df, column):
    
    # Dictionary where keys are tuples (year, author)
    # and values is number of record for the author in that year
    count = {}
    
    # Iterate rows in DataFrame
    for _, row in df.iterrows():
        year = row['year']
        for element in row[column]:
            key = (year, element) 
            # If key exists, add one
            if key in count.keys():
                count[key] += 1
            else:
                # Key does not exist, create key and add one
                count[key] = 1  
    return count              

# Function that counts the number of records in each year for each author
def find_most_common_multiple(count):
    
    # Dictionary, where keys are years
    # and values are lists of (author, number of records)
    most_common = {}

    # Iterate through keys and values in dictionary
    for key, value in count.items():
        year = key[0]

        # If key (year) exists
        if year in most_common:

            # Check if value (number of records) is higher than our saved one
            if most_common[year][0][1] < value:
                
                # Create a list with one value
                most_common[year] = [(key[1], value)]    
            
            # If the value equals the one we saved
            elif most_common[year][0][1] == value:
                # Append value to an existing list
                most_common[year].append((key[1], value))  
        else:
            # Remember the first value
            most_common[year] = [(key[1], value)] 
    
    # Sort keys in ascending order         
    years = list(most_common.keys())
    years.sort()
    sorted_most_common = {i: most_common[i] for i in years}        
    return sorted_most_common            

print("Functions saved.")


In [ ]:
# Select column 
picked_column = 'author'

# Call our functions
count = count_years(df, picked_column)

most_common = find_most_common_multiple(count)

# Format string
string = "{year: >25} {name: >25} {records: >25}"

# Print the most common authors
print(string.format(year = "Year", name = "Most common authors", records = "Number of records"))
print("----------------------------------------------------------------------------------------------------")
for key, value in most_common.items():
    for v in value:
        print(string.format(year = key, name = v[0], records = v[1]))  

<div class='alert alert-block alert-info'>
    <b>Try It!</b> Try other bibliographies as well. Just create a CSV table in the previous Jupyter notebook.
</div>
